In [3]:
import os
os.chdir('/Users/ritwikdeshpande/DLH/Project/CS598_DLH_Project/')
import weka.core.jvm as jvm
import numpy as np
import csv
import pandas as pd
from weka.classifiers import Classifier
from weka.core.converters import Loader
from weka.core.converters import Loader
from weka.classifiers import Evaluation
from weka.core.classes import Random
from dataset.preprocessing.tf_idf_all_feature_matrix_gen import TFIDFFeatureGeneration


jvm.start()
loader = Loader(classname="weka.core.converters.ArffLoader")


INFO:weka.core.jvm:JVM already running, call jvm.stop() first


In [12]:
class JRip:
    def __init__(self, train_data):
        self.jRip = Classifier(classname="weka.classifiers.rules.JRip")
        self.jRip.build_classifier(train_data)
        self.evaluator = Evaluation(train_data)

        
        # print(self.x_train.shape, self.y_train.shape, self.x_test.shape, self.y_test.shape)
    
    def test_and_evaluate(self, train_data):
        self.evaluator.crossvalidate_model(self.jRip, train_data, 10, Random(1))
        f1_macro = self.evaluator.weighted_f_measure
        f1_micro = self.evaluator.f_measure(1)
        # print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")
        return f1_macro, f1_micro

In [7]:
morbidities = ['Asthma', 'CAD', 'CHF', 'Depression', 'Diabetes', 'Gallstones', 'GERD', 'Gout', 'Hypercholesterolemia', 'Hypertension', 'Hypertriglyceridemia', 'OA', 'Obesity', 'OSA', 'PVD', 'Venous-Insufficiency']

column_headings = ["Morbidity Class", "DT_Macro F1", "DT_Micro F1"]

with open("./results/performance_JRip.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    

In [13]:
all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    f1_macro_list = []
    f1_micro_list = []
    train_data = loader.load_file(f"./dataset/train/train_{morbidity}_tfidf.arff")
    train_data.class_is_last()
    try:
        jRip_obj = JRip(train_data)
        f1_macro, f1_micro = jRip_obj.test_and_evaluate(train_data)
    except Exception as ex:
        print(str(ex))
        f1_macro = 1
        f1_micro = 1
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/performance_JRip.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)


with open("./results/performance_JRip.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Asthma
Macro F1 score: 0.9825174825174825 and Micro F1 Score 0.9285714285714286
CAD
Macro F1 score: 0.8987259711830168 and Micro F1 Score 0.9180327868852459
CHF
weka.classifiers.rules.JRip: Cannot handle unary class!
Macro F1 score: 1 and Micro F1 Score 1
Depression


Exception in thread "Thread-1" weka.core.UnsupportedAttributeTypeException: weka.classifiers.rules.JRip: Cannot handle unary class!
	at weka.core.Capabilities.test(Capabilities.java:1045)
	at weka.core.Capabilities.test(Capabilities.java:1256)
	at weka.core.Capabilities.test(Capabilities.java:1138)
	at weka.core.Capabilities.testWithFail(Capabilities.java:1468)
	at weka.classifiers.rules.JRip.buildClassifier(JRip.java:1667)


Macro F1 score: 0.8884517586825207 and Micro F1 Score 0.6974358974358973
Diabetes
Macro F1 score: 0.9033889585504628 and Micro F1 Score 0.9301143583227447
Gallstones
Macro F1 score: 0.7979010603679358 and Micro F1 Score 0.12962962962962962
GERD
Macro F1 score: 0.7096519844748265 and Micro F1 Score 0.26347305389221554
Gout
Macro F1 score: 0.8168547115940944 and Micro F1 Score 0.1509433962264151
Hypercholesterolemia
Macro F1 score: 0.8625372558882932 and Micro F1 Score 0.8622754491017964
Hypertension
Macro F1 score: 0.9298977954416917 and Micro F1 Score 0.9549763033175356
Hypertriglyceridemia
Macro F1 score: 0.9187078159976607 and Micro F1 Score 0.05555555555555555
OA
Macro F1 score: 0.7878176793093205 and Micro F1 Score 0.2857142857142857
Obesity
Macro F1 score: 0.9655100837411553 and Micro F1 Score 0.9591397849462366
OSA
Macro F1 score: 0.9359083146599805 and Micro F1 Score 0.7764705882352941
PVD
Macro F1 score: 0.930035164910846 and Micro F1 Score 0.7612903225806451
Venous-Insufficien

In [14]:
jvm.stop()